In [8]:
import torch
import torch_geometric
from torch_geometric.data import Data, DataLoader
import deepchem as dc
from rdkit import Chem
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data, InMemoryDataset
import multiprocessing
   
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv , TransformerConv
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [9]:
from torch_geometric.data import InMemoryDataset, Data

class DeepChemToPyGDataset(InMemoryDataset):
    def __init__(self, data_list, transform=None):
        """
        Converts a list of PyG `Data` objects into an InMemoryDataset.
        
        Args:
            data_list (list of Data): List of PyG Data objects.
            transform (callable, optional): Optional transform applied to each data object.
        """
        super(DeepChemToPyGDataset, self).__init__('.', transform, None, None)
        self.data, self.slices = self.collate(data_list)

    def get(self, idx):
        """
        Retrieves the data object at the specified index.
        """
        return super().get(idx)


In [10]:
pyg_dataset = torch.load(r'../../data/curated dataset/cleaned_frag_pyg_dataset.pth')
dataset=pyg_dataset

C:\Users\sarab\AppData\Local\Temp\ipykernel_30428\137928901.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pyg_dataset = torch.load(r'../../data/curated dataset/cleaned

In [11]:
# def load_data(file_path):
#     pyg_dataset = torch.load(r'C:\Users\sarab\OneDrive\Desktop\CSIR\generative_model_compare\Convertors\cleaned_frag_pyg_dataset.pth')
#     dataset_length = len(pyg_dataset)

#     # 80% training, 20% validation split
#     train_size = int(0.8 * dataset_length)
#     val_size = dataset_length - train_size
#     train_dataset, val_dataset = torch.utils.data.random_split(
#         pyg_dataset, [train_size, val_size]
#     )
#     return train_dataset, val_dataset, pyg_dataset[0].num_features, pyg_dataset[0].edge_attr.size(-1)

In [12]:
node_features = pyg_dataset[0].x.shape[1]  # Should be 134 based on your description
edge_features = pyg_dataset[0].edge_attr.shape[1]  # Should be 6 based on your description


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class ARGVA(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(ARGVA, self).__init__()
        # Initialize with correct input dimensions
        self.encoder = VariationalEncoder(node_features, edge_features, hidden_dim, latent_dim)
        self.decoder = Decoder(latent_dim, node_features, edge_features)
        self.discriminator = Discriminator(latent_dim)
        
    def encode(self, x, edge_index, edge_attr, batch=None):
        return self.encoder(x, edge_index, edge_attr, batch)

    def decode(self, z, edge_index, batch=None):
        return self.decoder(z, edge_index, batch)

    def discriminate(self, z):
        return self.discriminator(z)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        return mu + torch.randn_like(std) * std if self.training else mu

    def forward(self, x, edge_index, edge_attr, batch=None):
        mu, logvar = self.encode(x, edge_index, edge_attr, batch)
        z = self.reparameterize(mu, logvar)
        return self.decode(z, edge_index, batch) + (mu, logvar)

    def generate_multiple_outputs(self, x, edge_index, edge_attr, num_samples=5, temperature=0.5):
        mu, logvar = self.encode(x, edge_index, edge_attr)
        std = torch.exp(0.5 * logvar)
        
        outputs = []
        for _ in range(num_samples):
            with torch.no_grad():
                z = mu + temperature * std * torch.randn_like(std)
                x_rec, edge_rec = self.decode(z, edge_index)
                outputs.append((
                    F.softmax(x_rec/temperature, dim=-1),
                    F.softmax(edge_rec/temperature, dim=-1),
                    edge_index
                ))
        return outputs

class VariationalEncoder(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(VariationalEncoder, self).__init__()
        # First GCN layer processes node features
        self.conv1 = GCNConv(node_features, hidden_dim)
        # Second GCN layer processes combined features
        self.conv2 = GCNConv(hidden_dim + edge_features, hidden_dim)
        # Final transformation to latent space
        self.mu = nn.Linear(hidden_dim, latent_dim)
        self.logvar = nn.Linear(hidden_dim, latent_dim)
        
    def forward(self, x, edge_index, edge_attr, batch=None):
        # Process node features first
        h = F.relu(self.conv1(x, edge_index))
        
        # Add edge features to node embeddings
        edge_feat = torch.zeros(h.size(0), edge_attr.size(1), device=x.device)
        edge_feat.index_add_(0, edge_index[0], edge_attr)
        
        # Combine node embeddings with edge features
        h = torch.cat([h, edge_feat], dim=-1)
        
        # Second convolution
        h = F.relu(self.conv2(h, edge_index))
        
        # Pool if batch is provided
        if batch is not None:
            h = global_mean_pool(h, batch)
        else:
            h = h.mean(dim=0, keepdim=True)
            
        return self.mu(h), self.logvar(h)

class Decoder(nn.Module):
    def __init__(self, latent_dim, node_features, edge_features):
        super(Decoder, self).__init__()
        self.node_out = nn.Linear(latent_dim, node_features)
        self.edge_out = nn.Linear(latent_dim * 2, edge_features)
        
    def forward(self, z, edge_index, batch=None):
        z = z[batch] if batch is not None else z.repeat(edge_index.max().item() + 1, 1)
        
        # Decode nodes
        node_pred = self.node_out(z)
        
        # Decode edges using concatenated node features
        edge_input = torch.cat([z[edge_index[0]], z[edge_index[1]]], dim=1)
        edge_pred = self.edge_out(edge_input)
        
        return node_pred, edge_pred

class Discriminator(nn.Module):
    def __init__(self, latent_dim):
        super(Discriminator, self).__init__()
        self.out = nn.Linear(latent_dim, 1)

    def forward(self, z):
        return torch.sigmoid(self.out(z))

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Fixed hyperparameter
epochs = 60

In [15]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.loader import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

def train_argva_with_early_stopping(
    model, train_dataset, val_dataset, num_epochs, batch_size, lr=1e-4, device='cuda', patience=10
):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=patience//2, verbose=True)
    
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None

    # Dynamic loss weights
    beta_start = 0.01  # KL weight start
    beta_end = 1.0    # KL weight end
    adv_weight = 0.1  # Adversarial weight  #0.1

    for epoch in range(num_epochs):
        model.train()
        total_loss = total_recon_loss = total_kl_loss = total_adv_loss = 0
        
        # KL annealing
        # beta = min(beta_start + (beta_end - beta_start) * epoch / (num_epochs // 2), beta_end)
        beta = 0.01  #0.01

        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            # Forward pass
            reconstructed_x, reconstructed_edge_attr, mu, logvar = model(
                batch.x, batch.edge_index, batch.edge_attr, batch.batch
            )

            # Reconstruction loss with edge importance
            recon_loss = (
                F.mse_loss(reconstructed_x, batch.x) * 1 +
                F.mse_loss(reconstructed_edge_attr, batch.edge_attr) * 1
            )

            # KL divergence
            kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

            # Adversarial component
            z = model.reparameterize(mu, logvar)
            real_z = torch.randn_like(z)
            real_scores = model.discriminate(real_z)
            fake_scores = model.discriminate(z)
            adv_loss = -torch.mean(torch.log(real_scores + 1e-8) + torch.log(1 - fake_scores + 1e-8))

            # Combined loss
            loss = recon_loss + beta * kl_loss + adv_weight * adv_loss

            if not torch.isnan(loss) and not torch.isinf(loss):
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()

                total_loss += loss.item()
                total_recon_loss += recon_loss.item()
                total_kl_loss += kl_loss.item()
                total_adv_loss += adv_loss.item()

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                reconstructed_x, reconstructed_edge_attr, mu, logvar = model(
                    batch.x, batch.edge_index, batch.edge_attr, batch.batch
                )
                val_recon_loss = F.mse_loss(reconstructed_x, batch.x) + F.mse_loss(reconstructed_edge_attr, batch.edge_attr)
                val_kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
                val_loss += (val_recon_loss + beta * val_kl_loss).item()

        val_loss /= len(val_loader)
        avg_loss = total_loss / len(train_loader)
        
        print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f} Val: {val_loss:.4f} Beta: {beta:.4f}')

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print('Early stopping triggered')
                break

        scheduler.step(val_loss)

    if best_model_state:
        model.load_state_dict(best_model_state)

    return model

In [16]:
import os
import json
import torch
import optuna
import multiprocessing
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.loader import DataLoader as PyGDataLoader
from torch.utils.data import random_split

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
import optuna
import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader as PyGDataLoader
from multiprocessing import cpu_count

def objective(trial):
    # Suggest hyperparameters
    pyg_dataset = torch.load(r'../../data/curated dataset/cleaned_frag_pyg_dataset.pth')
    node_features = pyg_dataset[0].x.size(1)
    edge_features = pyg_dataset[0].edge_attr.size(1)
    hidden_dim = trial.suggest_int("hidden_dim", 64, 512, step=64)
    latent_dim = trial.suggest_int("latent_dim", 32, 128, step=32)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])
    patience = trial.suggest_int("patience", 3, 10)
    adv_weight = 0.1
    print(f"\n[Trial {trial.number}] Starting with parameters: {trial.params}")
    # Initialize model
    
    model = ARGVA(node_features, edge_features, hidden_dim, latent_dim)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_ratio = 0.8
    valid_ratio = 0.2
    beta = 0.01 
    
    train_ratio = 0.8
    num_train = int(len(pyg_dataset) * train_ratio)
    num_valid = len(pyg_dataset) - num_train
    train_dataset, valid_dataset = random_split(pyg_dataset, [num_train, num_valid])

    # Model initialization
    
    # Train and validate model
    trained_model = train_argva_with_early_stopping(
        model=model,
        train_dataset=train_dataset,
        val_dataset=valid_dataset,
        num_epochs=60,  # Adjust based on requirements
        batch_size=batch_size,
        lr=learning_rate,
        device=device,
        patience=patience
    )
        

    # Create DataLoader for training dataset
    train_loader = PyGDataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Create DataLoader for validation dataset
    val_loader = PyGDataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    # Validation loss as the optimization target
    # trained_model = train_argva_with_early_stopping(
    #     model=model,
    #     train_dataset=train_dataset,
    #     val_dataset=valid_dataset,
    #     num_epochs=60,  # Adjust based on requirements
    #     batch_size=batch_size,
    #     lr=learning_rate,
    #     device=device,
    #     patience=patience,
    #     adv_weight=adv_weight  # Pass adversarial weight to the training function
    # )

    # Validation loss as the optimization target
    val_loss = 0
    model.eval()
    with torch.no_grad():
        for batch in val_loader:
            batch = batch.to(device)
            reconstructed_x, reconstructed_edge_attr, mu, logvar = model(
                batch.x, batch.edge_index, batch.edge_attr, batch.batch
            )
            val_recon_loss = F.mse_loss(reconstructed_x, batch.x) + F.mse_loss(reconstructed_edge_attr, batch.edge_attr)
            val_kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
            z = trained_model.reparameterize(mu, logvar)
            real_z = torch.randn_like(z)
            real_scores = trained_model.discriminate(real_z)
            fake_scores = trained_model.discriminate(z)
            adv_loss = -torch.mean(torch.log(real_scores + 1e-8) + torch.log(1 - fake_scores + 1e-8))

            val_loss += (val_recon_loss + beta *val_kl_loss + adv_weight * adv_loss).item()

    val_loss /= len(val_loader)
    return val_loss

if __name__ == "__main__":
    # Create Optuna study
    
    num_jobs = min(cpu_count(), 27)  # Use up to 4 cores or fewer if limited
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=200, n_jobs=num_jobs)

    # Output the best parameters along with the trial number
    best_trial = study.best_trial
    print(f"\nBest Trial Number: {best_trial.number}")
    print("Best Parameters:", best_trial.params)
    print("Best Value (Objective):", best_trial.value)


[I 2025-01-12 00:57:39,100] A new study created in memory with name: no-name-450ef3e5-106f-4078-ada4-6b592d4de150


C:\Users\sarab\AppData\Local\Temp\ipykernel_30428\535849477.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pyg_dataset = torch.load(r'../../data/curated dataset/cleane


[Trial 1] Starting with parameters: {'hidden_dim': 256, 'latent_dim': 128, 'learning_rate': 0.04997842549958054, 'batch_size': 64, 'patience': 10}

[Trial 5] Starting with parameters: {'hidden_dim': 384, 'latent_dim': 96, 'learning_rate': 0.08710140797905021, 'batch_size': 32, 'patience': 7}

[Trial 9] Starting with parameters: {'hidden_dim': 256, 'latent_dim': 128, 'learning_rate': 0.005555908264553736, 'batch_size': 8, 'patience': 9}

[Trial 13] Starting with parameters: {'hidden_dim': 192, 'latent_dim': 32, 'learning_rate': 0.09709893341602743, 'batch_size': 8, 'patience': 6}

[Trial 15] Starting with parameters: {'hidden_dim': 192, 'latent_dim': 96, 'learning_rate': 0.032947189184635385, 'batch_size': 8, 'patience': 9}

[Trial 2] Starting with parameters: {'hidden_dim': 448, 'latent_dim': 96, 'learning_rate': 0.005216984374571024, 'batch_size': 16, 'patience': 9}

[Trial 10] Starting with parameters: {'hidden_dim': 192, 'latent_dim': 128, 'learning_rate': 0.006518608445872273, 'ba

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/60] Loss: 1.1128 Val: 0.3029 Beta: 0.0100
Epoch [1/60] Loss: 0.9070 Val: 0.1393 Beta: 0.0100
Epoch [1/60] Loss: 36.2629 Val: 0.1376 Beta: 0.0100
Epoch [1/60] Loss: 0.2406 Val: 0.1056 Beta: 0.0100
Epoch [1/60] Loss: 1.0369 Val: 0.2018 Beta: 0.0100
Epoch [1/60] Loss: 0.2492 Val: 0.1052 Beta: 0.0100
Epoch [1/60] Loss: 1007163812471776896.0000 Val: inf Beta: 0.0100
Epoch [1/60] Loss: 0.1977 Val: 0.1014 Beta: 0.0100
Epoch [1/60] Loss: 0.5693 Val: 0.1080 Beta: 0.0100
Epoch [2/60] Loss: 1.0763 Val: 0.2875 Beta: 0.0100
Epoch [2/60] Loss: 0.5869 Val: 0.1170 Beta: 0.0100
Epoch [2/60] Loss: 0.1494 Val: 0.1008 Beta: 0.0100
Epoch [2/60] Loss: 0.1412 Val: 0.1122 Beta: 0.0100
Epoch [2/60] Loss: 0.8055 Val: 0.1578 Beta: 0.0100
Epoch [2/60] Loss: 0.1589 Val: 0.0994 Beta: 0.0100
Epoch [2/60] Loss: 0.0000 Val: inf Beta: 0.0100
Epoch [1/60] Loss: 0.2292 Val: 0.1063 Beta: 0.0100
Epoch [3/60] Loss: 1.0474 Val: 0.2736 Beta: 0.0100
Epoch [3/60] Loss: 0.3105 Val: 0.1083 Beta: 0.0100
Epoch [1/60] Loss:

...

Epoch [57/60] Loss: 0.1000 Val: 0.0939 Beta: 0.0100
Epoch [44/60] Loss: 0.0963 Val: 0.0956 Beta: 0.0100
Epoch [58/60] Loss: 0.1003 Val: 0.0937 Beta: 0.0100
Epoch [45/60] Loss: 0.0963 Val: 0.0956 Beta: 0.0100
Epoch [59/60] Loss: 0.1000 Val: 0.0932 Beta: 0.0100
Epoch [46/60] Loss: 0.0966 Val: 0.0962 Beta: 0.0100
Epoch [60/60] Loss: 0.1006 Val: 0.0933 Beta: 0.0100


[I 2025-01-12 13:02:18,555] Trial 194 finished with value: 0.09356996730095198 and parameters: {'hidden_dim': 256, 'latent_dim': 128, 'learning_rate': 0.02225508218201987, 'batch_size': 8, 'patience': 8}. Best is trial 72 with value: 0.08646482492194456.


Epoch [47/60] Loss: 0.0963 Val: 0.0955 Beta: 0.0100
Epoch [48/60] Loss: 0.0947 Val: 0.0951 Beta: 0.0100
Epoch [49/60] Loss: 0.0953 Val: 0.0950 Beta: 0.0100
Epoch [50/60] Loss: 0.0952 Val: 0.0952 Beta: 0.0100
Epoch [51/60] Loss: 0.0951 Val: 0.0953 Beta: 0.0100
Epoch [52/60] Loss: 0.0949 Val: 0.0951 Beta: 0.0100
Epoch [53/60] Loss: 0.0949 Val: 0.0951 Beta: 0.0100
Epoch [54/60] Loss: 0.0950 Val: 0.0951 Beta: 0.0100
Epoch [55/60] Loss: 0.0947 Val: 0.0949 Beta: 0.0100
Epoch [56/60] Loss: 0.0945 Val: 0.0950 Beta: 0.0100
Epoch [57/60] Loss: 0.0944 Val: 0.0950 Beta: 0.0100
Epoch [58/60] Loss: 0.0944 Val: 0.0951 Beta: 0.0100
Epoch [59/60] Loss: 0.0947 Val: 0.0950 Beta: 0.0100
Epoch [60/60] Loss: 0.0945 Val: 0.0950 Beta: 0.0100


[I 2025-01-12 13:03:36,256] Trial 195 finished with value: 0.09562067678119957 and parameters: {'hidden_dim': 448, 'latent_dim': 32, 'learning_rate': 0.023914510778901134, 'batch_size': 8, 'patience': 8}. Best is trial 72 with value: 0.08646482492194456.



Best Trial Number: 72
Best Parameters: {'hidden_dim': 384, 'latent_dim': 128, 'learning_rate': 0.018061505448184818, 'batch_size': 8, 'patience': 8}
Best Value (Objective): 0.08646482492194456
